In [35]:
print("OK")

OK


In [36]:
%pwd

'/Users/karthickpm/Desktop'

In [37]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Keep 'source' AND 'page' in metadata.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        # Extract source and page
        src = doc.metadata.get("source")
        # Get page number, default to 0 if missing
        page = doc.metadata.get("page", 0)

        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                # We now include 'page' in the metadata so it saves to Pinecone
                metadata={"source": src, "page": page}
            )
        )
    return minimal_docs

In [30]:
import os 
os.chdir("../")

In [31]:
%pwd

'/Users/karthickpm/Desktop'

In [32]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [34]:
extracted_data = load_pdf_files("data")

FileNotFoundError: Directory not found: 'data'

In [9]:
extracted_data

[Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-11-12T04:26:35+00:00', 'author': 'roma.farooq', 'moddate': '2025-11-12T04:26:35+00:00', 'source': 'data/003-BPZ#1 Technical Report.pdf', 'total_pages': 170, 'page': 0, 'page_label': '1'}, page_content='1 \n  \n \n \n \nThe Terra Mater Complex \n11th December 2024 \n \n \n \nReport By \nRoma Farooq, Marjorie Chamberlain, Meghana Gali, Kwarteng Frimpong \n \nClient’s Name \nDr. Xiaohong Chen'),
 Document(metadata={'producer': 'PyPDF', 'creator': 'Microsoft Word', 'creationdate': '2025-11-12T04:26:35+00:00', 'author': 'roma.farooq', 'moddate': '2025-11-12T04:26:35+00:00', 'source': 'data/003-BPZ#1 Technical Report.pdf', 'total_pages': 170, 'page': 1, 'page_label': '2'}, page_content='2 \nAcknowledgements \nWe would like to express our sincere gratitude to Northumbria University, Newcastle, for providing \nus with the invaluable opportunity to engage in advanced practice, which has greatly enhanced 

In [10]:
len(extracted_data)

319

In [11]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [13]:
minimal_docs

[Document(metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='1 \n  \n \n \n \nThe Terra Mater Complex \n11th December 2024 \n \n \n \nReport By \nRoma Farooq, Marjorie Chamberlain, Meghana Gali, Kwarteng Frimpong \n \nClient’s Name \nDr. Xiaohong Chen'),
 Document(metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='2 \nAcknowledgements \nWe would like to express our sincere gratitude to Northumbria University, Newcastle, for providing \nus with the invaluable opportunity to engage in advanced practice, which has greatly enhanced our \nexposure, confidence, and knowledge in our field. A special thank you to Dr. Xiaohong Chen for \nselecting us as part of her team and giving us the privilege to collaborate on this project. Her \nguidance has been instrumental in our development. We are also grateful to John Arthur for \nimparting his expertise on the art of professional presentation, significantly improving the way we \npresent our work. \nOur

In [14]:
# Split the documents into smaller chunks
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [15]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 1067


In [16]:
texts_chunk

[Document(metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='1 \n  \n \n \n \nThe Terra Mater Complex \n11th December 2024 \n \n \n \nReport By \nRoma Farooq, Marjorie Chamberlain, Meghana Gali, Kwarteng Frimpong \n \nClient’s Name \nDr. Xiaohong Chen'),
 Document(metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='2 \nAcknowledgements \nWe would like to express our sincere gratitude to Northumbria University, Newcastle, for providing \nus with the invaluable opportunity to engage in advanced practice, which has greatly enhanced our \nexposure, confidence, and knowledge in our field. A special thank you to Dr. Xiaohong Chen for \nselecting us as part of her team and giving us the privilege to collaborate on this project. Her'),
 Document(metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='guidance has been instrumental in our development. We are also grateful to John Arthur for \nimparting his expertise on the art of pr

In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

/var/folders/kv/dkdwr9cx07vgwzpzj4j3wqx40000gn/T/ipykernel_60116/171795349.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [18]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
vector = embedding.embed_query("Hello world")
vector

[-0.03447728604078293,
 0.031023193150758743,
 0.006734972819685936,
 0.026109015569090843,
 -0.03936203196644783,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441446952521801,
 -0.047450508922338486,
 0.014758873730897903,
 0.07087536156177521,
 0.05552757531404495,
 0.019193297252058983,
 -0.026251330971717834,
 -0.01010951865464449,
 -0.026940520852804184,
 0.02230742573738098,
 -0.02222665771842003,
 -0.14969265460968018,
 -0.017493057996034622,
 0.007676220964640379,
 0.054352302104234695,
 0.0032544543500989676,
 0.03172598406672478,
 -0.08462141454219818,
 -0.029405983164906502,
 0.05159566551446915,
 0.04812401905655861,
 -0.0033147598151117563,
 -0.05827919393777847,
 0.04196927696466446,
 0.022210730239748955,
 0.128188818693161,
 -0.022338904440402985,
 -0.011656290851533413,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122607111930847,
 -0.031175294890999794,
 0.05269956961274147,
 0.04703482985496521,
 -0.08420304954051971,
 -0.030056187883019447,
 -0.02074483

In [20]:
print( "Vector length:", len(vector))

Vector length: 384


In [21]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [22]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [23]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [24]:
pc

In [25]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from typing import List
from langchain.schema import Document

load_dotenv()

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "bpz-chatbot"

In [26]:
from pinecone import ServerlessSpec 

index_name = "bpz-chatbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [27]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

KeyboardInterrupt: 

In [30]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

# Add more data to the existing Pinecone index

In [31]:
dswith = Document(
    page_content="dswithbappy is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [32]:
docsearch.add_documents(documents=[dswith])

['182b690b-6601-469b-a708-6924759f9fa9']

In [33]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [34]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='4ddf179a-62fc-487a-aa2b-012940fecb50', metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='appearance in a residential setting.  Technology has improved since 2011 which may enable a'),
 Document(id='5bfff47c-daa1-4078-831a-6977a3f7b585', metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='appearance in a residential setting.  Technology has improved since 2011 which may enable a'),
 Document(id='528a1442-ca8d-4106-9eb9-a2303c65e28f', metadata={'source': 'data/003-BPZ#1 Technical Report.pdf'}, page_content='Polysolar PS-CT Series can be found in appendix 5.')]

In [35]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o")

In [36]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [38]:
system_prompt = (
    "You are an senior construction assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [39]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [40]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly and gigantism are disorders caused by the excessive production of growth hormone, usually due to a pituitary gland tumor. Acromegaly occurs in adults and results in enlarged bones and tissues, particularly in the hands, feet, and face. Gigantism occurs in children or adolescents whose growth plates have not yet closed, leading to excessive growth in height.


In [41]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

Acne is a common skin condition characterized by the presence of pimples, blackheads, and cysts, often caused by clogged hair follicles, excess oil production, bacteria, and inflammation. It primarily affects the face, shoulders, back, neck, and chest. Acne is particularly common during adolescence but can occur at any age.


In [42]:
response = rag_chain.invoke({"input": "what is the Treatment of Acne?"})
print(response["answer"])

I don't know.


In [40]:
response = rag_chain.invoke({"input": "what dswithbappy?"})
print(response["answer"])

dswithbappy is a YouTube channel that provides tutorials on various topics.


In [42]:
import os
import time
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# 1. SETUP & CONFIGURATION
load_dotenv()

# --- 🟢 UPDATED PATH: POINTING DIRECTLY TO YOUR FOLDER ---
DATA_PATH = "/Users/karthickpm/Desktop/Build planet zero1/data"
INDEX_NAME = "bpz-chatbot"

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("❌ Error: Missing PINECONE_API_KEY in .env file")

print(f"📍 Target Data Folder: {DATA_PATH}")

# 2. LOAD PDFS
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ Error: The folder '{DATA_PATH}' does not exist.")

print("📂 Loading PDFs...")
loader = DirectoryLoader(DATA_PATH, glob="*.pdf", loader_cls=PyPDFLoader)
raw_docs = loader.load()

if not raw_docs:
    print("❌ Error: No PDF files found in the folder.")
    exit()

print(f"✅ Loaded {len(raw_docs)} pages.")

# 3. CLEAN METADATA (Preserve Page Numbers)
print("🧹 Cleaning metadata...")
cleaned_docs = []
for doc in raw_docs:
    # Use the filename as source, not the full absolute path (cleaner for frontend)
    full_path = doc.metadata.get("source", "")
    filename = os.path.basename(full_path)
    page = doc.metadata.get("page", 0)

    cleaned_docs.append(
        Document(
            page_content=doc.page_content,
            metadata={"source": filename, "page": page}
        )
    )

# 4. SPLIT TEXT
print("✂️  Splitting text...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
chunks = text_splitter.split_documents(cleaned_docs)
print(f"✅ Created {len(chunks)} text chunks.")

# 5. INITIALIZE EMBEDDINGS
print("🧠 Loading Embeddings Model...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 6. RESET PINECONE INDEX
print("🌲 Connecting to Pinecone...")
pc = Pinecone(api_key=PINECONE_API_KEY)

# List existing indexes
existing_indexes = [i.name for i in pc.list_indexes()]

if INDEX_NAME in existing_indexes:
    print(f"🗑️  Deleting old index '{INDEX_NAME}'...")
    pc.delete_index(INDEX_NAME)
    # Wait loop to ensure deletion is registered
    while INDEX_NAME in [i.name for i in pc.list_indexes()]:
        time.sleep(1)

print(f"🔨 Creating new index '{INDEX_NAME}'...")
pc.create_index(
    name=INDEX_NAME,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

# Wait for index to be ready
while not pc.describe_index(INDEX_NAME).status['ready']:
    time.sleep(1)

# 7. UPLOAD
print("🚀 Uploading vectors to Pinecone...")
PineconeVectorStore.from_documents(chunks, embeddings, index_name=INDEX_NAME)

print("\n🎉 SUCCESS! All documents processed and uploaded.")

📍 Target Data Folder: /Users/karthickpm/Desktop/Build planet zero1/data
📂 Loading PDFs...
✅ Loaded 319 pages.
🧹 Cleaning metadata...
✂️  Splitting text...
✅ Created 1067 text chunks.
🧠 Loading Embeddings Model...
🌲 Connecting to Pinecone...
🗑️  Deleting old index 'bpz-chatbot'...
🔨 Creating new index 'bpz-chatbot'...
🚀 Uploading vectors to Pinecone...

🎉 SUCCESS! All documents processed and uploaded.
